In [4]:
"""
Authors: Rahul Sawhney, Nikhil Kumar Pradhan, Amit Kumar
Project Control-Flow: 1) class Dataset
                                : __init__
                                : __len__
                                : __getitem__
                                :@ normalize
                                :@ get_data
                      
                      2) class DataAnalysis:
                                :@ One Batch pics Visualization 
                                :@ bar plot b/t all the tumors 
                                :@ Images after applying 7-8 different convolutional kernels
                                :@ ...  
                      
                      3) class Preprocess
                                :@ Resize
                                :@ RandomCrop
                                :@ RandomHorizontalFlip
                                :@ Normalize and denormalize to Tensors
                                :@ Arguments all the images to generate similar images

                      4) class GPU_Acceleration
                                :@ hardware_acceleration
                                :@ set_GPU
                                :@ load_GPU
                                :@ is_working_GPU
                                : __init__
                                : __len__
                                : __iter__

                      5) class Hyperparams
                                : >Layers 
                                : >Neurons Engineering
                                : >Activation Functions 
                      
                      6) class CNNModel
                                :__init__
                                :@ forward
                      
                      7) class TrainCNN
                                : train_dataset --> Training Dir...
                      
                      8) class TestCNN
                                : Model on New unseen Data --> Testing Dir ...
                      
                      9) class EvaluateCNN
                                :@ plot_1: Accuracy vs No. of epochs
                                :@ plot_2: Training and Validation Loss
                                :@ plot_3: Learning Rate vs Batch no
                                :@ plot_4: predict() -> make prediction on 1 image and show that image
                                :@ plot_5: ...
                                :@ plot_6: ...
                      
                      10) class SaveCNN := BrainMRI_MODEL.pt
                                :@ save_CNN
                                :@ is_save_CNN
                                :@ load_CNN
                                :@ is_load_CNN

Dataset Link: https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri


Project Abstract:
A Brain tumor is considered as one of the aggressive diseases, among children and adults. Brain tumors account for 85 to 90 percent of all primary Central Nervous 
System(CNS) tumors. Every year, around 11,700 people are diagnosed with a brain tumor. The 5-year survival rate for people with a cancerous brain or CNS tumor is 
approximately 34 percent for men and36 percent for women. Brain Tumors are classified as: Benign Tumor, Malignant Tumor, Pituitary Tumor, etc. 
Proper treatment, planning, and accurate diagnostics should be implemented to improve the life expectancy of the patients. The best technique to 
detect brain tumors is Magnetic Resonance Imaging (MRI). A huge amount of image data is generated through the scans. These images are examined by the radiologist. 
A manual examination can be error-prone due to the level of complexities involved in brain tumors and their properties.
Application of automated classification techniques using Machine Learning(ML) and Artificial Intelligence(AI)has consistently shown higher accuracy than manual 
classification. Hence, proposing a system performing detection and classification by using Deep Learning Algorithms using ConvolutionNeural Network (CNN), 
Artificial Neural Network (ANN), and TransferLearning (TL) would be helpful to doctors all around the world.


Context:
Brain Tumors are complex. There are a lot of abnormalities in the sizes and location of the brain tumor(s). This makes it really difficult for complete 
understanding of the nature of the tumor. Also, a professional Neurosurgeon is required for MRI analysis. Often times in developing countries the lack of
skillful doctors and lack of knowledge about tumors makes it really challenging and time-consuming to generate reports from MRI’. So an automated system on Cloud 
can solve this problem.
                      
"""

# py imports
from __future__ import annotations
import typing
typing.__name__ = "__main_module__"
from typing import Any, NewType, Generator, Optional, Union
import os, warnings
warnings.filterwarnings(action= 'ignore')


# Data Analysis Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


# Scripting ML 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


# DL imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary



# torch typing scripts
_path =  NewType("_path", Any)
_transform = NewType("_transform", Any)
_img = NewType("_img", Any)
_criterion = NewType("_criterion", Any)
_optimizor = NewType("_optimizor", Any)
_loss = NewType("_loss", Any)
_layer = NewType("_layer", Any)
_activation = NewType("_activation", Any)
_text = NewType("_text", Any)
_plot = NewType("_plot", Any)
_loader = NewType("_loader", Any)


#...DIR ...
# Testing:    
#        glioma_tumor 
#        meningioma_tumor 
#        no_tumor
#        pituitary_tumor
#
# Training: 
#        glioma_tumor 
#        meningioma_tumor 
#        no_tumor
#        pituitary_tumor
#...

# Class batches DataLoader Iterator:
#@ Unit test Pass  
class BrainMRIDataset(Dataset):
    '''
        A Custom `DataLoader Iterator` Class to load the set of Images in a `batch` 
        and applies some kind of `transformation` to every set of image in a batch of `Iterators`.

        INIT Args:
            path: _path     = Path of the Image Dataset folder
            sub_path: str   = str <Training | Testing | Validation>
            
        OPTIONAL Args:
            batch_size: int = size of the batches in the Training and Testing Dataset
            img_resolution: Optional[int]   = resolution of the image to load. :Options = [8, 16, 32, 64, 128] pixels
            transform: Optional[_transform] = `Sequencial Container` of various feature engineering teachiques.
        
        EXAMPLES:
            >>> df_train: pd.DataFrame = BRAINDataset(path= .../BRAIN_TUMOR_MRI/, 
                                                      sub_path= "Training",
                                                      batch_size= 8,
                                                      img_resolution= 32,
                                                      transform= None)
            
            >>> df_test: pd.DataFrame = BRAINDataset(path= .../BRAIN_TUMOR_MRI/,
                                                     sub_path= "Testing",
                                                     batch_size= 1,
                                                     img_resolution= 32,
                                                     transform= None)


        UNIT TEST: 
            >>> path: _path = "C:\\Users\\Lenovo\\OneDrive\\Desktop\\BRIAN_unit_testing\\"
            >>> sub_path: list[str] = ["Testing", "Training"]

            >>> df_train = BrainMRIDataset(path= path, 
                                           sub_path= sub_path[1],
                                           batch_size= 8)
    
            >>> df_test = BrainMRIDataset(path= path, 
                                          sub_path= sub_path[0],
                                          batch_size= 1)
    
            >>> print(df_test)

    '''
    def __init__(self, path: _path, sub_path: str, batch_size: Optional[int] = 4, img_resolution: Optional[int] = 64, transform: Optional[_transform] = None) -> None:
        self.path = path
        self.sub_path = sub_path
        self.batch_size = batch_size
        self.img_resolution = img_resolution
        if transform:
            self.transform = transform
        self.categories: list[str] = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]

        if sub_path == 'Training':
            self.dataset: pd.DataFrame = self.get_data(path, 'Training', self.categories)
        if sub_path == 'Testing':
            self.dataset: pd.DataFrame = self.get_data(path, 'Testing', self.categories)
        
        indexes: list[int] = [x for x in range(len(self.dataset))]
        
        self.index_batch: list[list[int]] = [
            indexes[i : i + batch_size]
            for i in range(0, len(indexes), batch_size)
        ]
        
    

    def __len__(self) -> tuple[int, ...]:
        return len(self.index_batch)
    

 
    def __getitem__(self, index: int) -> dict[_img, str]:
        batch: list[int] = self.index_batch[index]
        size: tuple[int, ...] = (self.img_resolution, self.img_resolution)
        images: list[int] = []
        labels: list[str] = []
        for i in batch:
            img: _img = Image.open(self.dataset.iloc[i].path).convert('LA').resize(size)
            img: np.ndarray = np.array(img)
            lbl: list[str] = [self.dataset.iloc[i].label]
            images.append(img)
            labels.append(np.array(lbl))
        
        images: torch.Tensor = torch.tensor(images).type(torch.float32)
        images: torch.Tensor = images.permute(0, 3, 1, 2)
        labels: torch.Tensor = torch.tensor(labels).type(torch.float32)
        images: torch.Tensor = self.normalize(images)
        return images, labels



    @classmethod
    def normalize(cls, x: torch.Tensor) -> torch.Tensor:
        return x / 255



    @classmethod
    def get_data(cls, path: _path, sub_path: str, categories: list[str]) -> pd.DataFrame:
        glioma_tumor: _path     = path + sub_path + "\\" + categories[0] + "\\"
        meningioma_tumor: _path = path + sub_path + "\\" + categories[1] + "\\"
        no_tumor: _path         = path + sub_path + "\\" + categories[2] + "\\"
        pituitary_tumor: _path  = path + sub_path + "\\" + categories[3] + "\\"

        glioma_tumor_list: list[str]  = [
            os.path.abspath(os.path.join(glioma_tumor, p))
            for p in os.listdir(glioma_tumor)                            
        ]
        
        meningioma_tumor_list: list[str] = [
            os.path.abspath(os.path.join(meningioma_tumor, p))
            for p in os.listdir(meningioma_tumor)
        ]

        no_tumor_list: list[str] = [
            os.path.abspath(os.path.join(no_tumor, p))
            for p in os.listdir(no_tumor)
        ]  
        

        pituitary_tumor_list: list[str] = [
            os.path.abspath(os.path.join(pituitary_tumor, p))
            for p in os.listdir(pituitary_tumor)
        ]

        glioma_tumor_labels: list[int]     = [0 for _ in range(len(glioma_tumor_list))]
        meningioma_tumor_labels: list[int] = [1 for _ in range(len(meningioma_tumor_list))]
        no_tumor_labels: list[int]         = [2 for _ in range(len(no_tumor_list))]
        pituitary_tumor_labels: list[int]  = [3 for _ in range(len(pituitary_tumor_list))]

        paths: _path      = glioma_tumor_list + meningioma_tumor_list + no_tumor_list + pituitary_tumor_list
        labels: list[int] = glioma_tumor_labels + meningioma_tumor_labels + no_tumor_labels + pituitary_tumor_labels

        dataframe: pd.DataFrame = pd.DataFrame.from_dict({'path': paths, 'label': labels})
        dataframe: pd.DataFrame = dataframe.sample(frac= 1)
        return dataframe 






#@ nimkhil @Amitwa
# Class Data Analysis
class BRAINAnalysis:
    def __repr__(self) -> tuple[str, ...]:
        return self.__module__, type(self).__name__, hex(id(self))



    def __str__(self) -> dict[str, str]:
        info: list[str] = ['module', 'name', 'ObjectID']
        return {item: value for item, value in zip(info, self.__repr__())} 



    @classmethod
    def plot(cls) -> _img:
        ...

    ...




#@ ... 
# Class Data Preprocess
class BRAINPreprocess:
    def __repr__(self) -> tuple[str, ...]:
        return self.__module__, type(self).__name__, hex(id(self))



    def __str__(self) -> dict[str, str]:
        info: list[str] = ['module', 'name', 'ObjectID']
        return {item: value for item, value in zip(info, self.__repr__())} 



    @classmethod
    def something(cls) -> Any:
        ...





# class GPU_acceleration
class GPU_Acceleration:
    def __init__(self, dataloader: _loader, device: Any) -> None:
        self.dataloader = dataloader
        self.device = device


    def __repr__(self) -> tuple[str, ...]:
        return self.__module__, type(self).__name__, hex(id(self))



    def __str__(self) -> dict[str, str]:
        info: list[str] = ['module', 'name', 'ObjectID']
        return {item: value for item, value in zip(info, self.__repr__())} 


    @classmethod
    def get_default_device(cls) -> str:
        if torch.cuda.is_available():
            return torch.device('cuda')
        return torch.device('cpu')
    

    @classmethod
    def to_device(cls, data: torch.Tensor, device: Any) -> Union[to_device, list]:
        if isinstance(data, (list, tuple)):
            return [to_device(x, device) for x in data]
        return data.to(device, non_blocking= True)
    

    def __len__(self) -> int:
        return len(self.dataloader)
    


    def __iter__(self) -> Generator[_loader, None, None]:
        for data in self.dataloader:
            yield self.to_device(data, self.device)
      


    @classmethod
    def is_working_GPU(cls) -> bool:
        return True if cls.get_default_device() == 'cuda' else False




# class Hyperparams
#@ Unit TEST: Passed 
class CNNHyperParams(object):
    '''
        Params:
            epochs: int           = No. of times the `Entire Dataset` is passed `Forward` abd `Backward` through the Neural Net
            criterion: _criterion = Criterion which `Optimizes` a Multi-class hinde loss 
            optimizor: _optimizor = ...
            learning_rate: float  = ...
            weight_decay: float   = ...
            momentum: float       = ...    

        DL Layers:
            convolution: _layer = Linear operation that involves the multiplication of a set of weights with the inpu
            pooling: _layer     = ...
            linear: _layer      = ...

        Neurons Engineering:
            flatten = ...
            dropout = ...

        Activation Functions:
            relu: _activation    = ...
            softmax: _activation = ...
            tanh: _activation    = ...

    '''
    # HyperParams: @params
class Hyperparams:
    convolutional_1 = nn.Conv2d(in_channels= 2, out_channels= 32, kernel_size= (3, 3))
    convolutional_2 = nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size= (3, 3))
    convolutional_3 = nn.Conv2d(in_channels= 64, out_channels= 128, kernel_size= (3, 3))
    convolutional_4 = nn.Conv2d(in_channels= 128, out_channels= 32, kernel_size= (3, 3))

    pooling_1 = nn.MaxPool2d(kernel_size= (2, 2), stride= 2)
    pooling_2 = nn.MaxPool2d(kernel_size= (2, 2), stride= 2)
    pooling_3 = nn.MaxPool2d(kernel_size= (2, 2), stride= 2)
    pooling_4 = nn.MaxPool2d(kernel_size= (2, 2), stride= 2)

                            # 512
    linear_1 = nn.Linear(in_features= 128, out_features= 64)
    linear_2 = nn.Linear(in_features= 64, out_features= 32)
    linear_3 = nn.Linear(in_features= 32, out_features= 16)
    linear_4 = nn.Linear(in_features= 16, out_features= 8)
    linear_5 = nn.Linear(in_features= 8, out_features= 4)

    relu = nn.ReLU()
    tanh = nn.Tanh()
    softmax = nn.Softmax()


    flatten = nn.Flatten()
    dropout = nn.Dropout(p= 0.3, inplace= False)
    

class CNN_Model(nn.Module):
    def __init__(self, num_classes: int) -> None:
        super(CNN_Model, self).__init__()
        self.convolution_layers = nn.Sequential(
            Hyperparams.convolutional_1,
            Hyperparams.tanh,
            Hyperparams.pooling_1,

            Hyperparams.convolutional_2,
            Hyperparams.tanh,
            Hyperparams.pooling_2,

            Hyperparams.convolutional_3,
            Hyperparams.relu,
            Hyperparams.pooling_3,

            Hyperparams.convolutional_4,
            Hyperparams.tanh,
            Hyperparams.pooling_4
            
        )
        self.linear_layers = nn.Sequential(
            Hyperparams.dropout,
            Hyperparams.linear_1,
            Hyperparams.tanh,

            Hyperparams.linear_2,
            Hyperparams.relu,

            Hyperparams.linear_3,
            Hyperparams.relu,

            Hyperparams.linear_4,
            Hyperparams.tanh,

            Hyperparams.linear_5,
        
        )

    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #print(x.shape)
        x: torch.Tensor = self.convolution_layers(x)
        #print(x.shape)            # 8, 2, 64, 64
        #x: torch.Tensor = x.view(-1, 4 * 4 * 32)
        #print(x.shape)
        x = Hyperparams.flatten(x)
        #print(x.shape)
        #print(x.view(x.size(0), -1).shape)
        #print(x.shape)
        x: torch.Tensor = self.linear_layers(x)

        #return Hyperparams.softmax(x)
        return x

# class train CNN
class TrainCNN(object):
    losses: list[float] = []

    def __repr__(self) -> tuple[str, ...]:
        return self.__module__, type(self).__name__, hex(id(self))



    def __str__(self) -> dict[str, str]:
        info: list[str] = ['module', 'name', 'ObjectID']
        return {item: value for item, value in zip(info, self.__repr__())} 



    @classmethod
    def train(cls, model: _model, epochs: int,  x_train: torch.Tensor, 
                   y_train: torch.Tensor, optimizor: _optimizor, criterion: _criterion ) -> _text:
        ...



    @classmethod
    def test(cls, model: _model, x_test: torch.Tensor, y_test: torch.Tensor, 
                  optimizor: _optimizor, criterion: _criterion) -> _text:
        ...



    def get_losses(self) -> list[float]:
        return self.losses





# CNN on test SET
class TestCNN:
    def __repr__(self) -> tuple[str, ...]:
        return self.__module__, type(self).__name__, hex(id(self))



    def __str__(self) -> dict[str, str]:
        info: list[str] = ['module', 'name', 'ObjectID']
        return {item: value for item, value in zip(info, self.__repr__())} 
    


    ...




# class Eval CNN
class EvaluateCNN:
    def __repr__(self) -> tuple[str, ...]:
        return self.__module__, type(self).__name__, hex(id(self))



    def __str__(self) -> dict[str, str]:
        info: list[str] = ['module', 'name', 'ObjectID']
        return {item: value for item, value in zip(info, self.__repr__())} 



    @classmethod
    def plot_1(cls) -> _plot:
        ...


    @classmethod
    def plot_2(cls) -> _plot:
        ...

   
    @classmethod
    def plot_3(cls) -> _plot:
        ...


    @classmethod
    def plot_4(cls) -> _plot:
        ...


    @classmethod
    def plot_5(cls) -> _plot:
        ...


    @classmethod
    def plot_6(cls) -> _plot:
        ...



# class Save CNN
# Trained Model Name := BrainMRI_MODEL.pt file
class SaveCNN:
    def __repr__(self) -> tuple[str, ...]:
        return self.__module__, type(self).__name__, hex(id(self))



    def __str__(self) -> dict[str, str]:
        info: list[str] = ['module', 'name', 'ObjectID']
        return {item: value for item, value in zip(info, self.__repr__())} 



    @classmethod
    def save(cls, model: _model) -> None:
        torch.save(model.state_dict(), 'BrainMRI_MODEL.pt')
    

    
    @classmethod
    def is_save_model(cls, model: _model) -> bool:
        ...


    @classmethod
    def load_model(cls, new_model: _model, old_model: str) -> _model:
        new_model.load_state_dict(torch.load(old_model))
        return new_model


    @classmethod
    def is_load_model(cls, new_model: _model, old_model: _model) -> bool:
        ...





# Driver code
if __name__ == "__main_module__":
    '''
    path: _path = ...
    sub_path: list[str] = ["Testing", "Training"]
    

    df_train: object = BrainMRIDataset(path= path, 
                                            sub_path= sub_path[1],
                                            batch_size= 6, 
                                            img_resolution= 64, 
                                            transforms= None)
    
    df_test: object = BrainMRIDataset(path= path, 
                                            sub_path= sub_path[0],
                                            batch_size= 6, 
                                            img_resolution= 64, 
                                            transforms= None)
    

    # train_loader: _loader = DataLoader(df_train, batch_size= 6, shuffle= True)
    # test_loader: _loader = DataLoader(df_test, batch_size= 1, shuffle= False)
    '''
    #model: _model = CNNModel()
    pass    












In [5]:
path: _path = "C:\\Users\\Lenovo\\OneDrive\\Desktop\\BRIAN_unit_testing\\"
sub_path: list[str] = ["Testing", "Training"]

df_train = BrainMRIDataset(path= path, 
                           sub_path= sub_path[1],
                           batch_size= 8)

df_test = BrainMRIDataset(path= path, 
                          sub_path= sub_path[0],
                          batch_size= 1)


             

In [6]:
model = CNN_Model(3)
print(model)

CNN_Model(
  (convolution_layers): Sequential(
    (0): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1))
    (10): Tanh()
    (11): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): Tanh()
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Linear(in_features=32, out_features=16, bias=True)
    (6)

In [7]:
img, lbls = df_train[0]


In [14]:
optimizer = torch.optim.SGD(model.parameters(), lr= 0.1)
criterion = nn.CrossEntropyLoss()

## Epochs and Final Trained Loss
* Epochs : 20
* Loss: 

In [9]:
mean_loss = []
for e in range(20):
    
    epoch_loss = []
    
    for idx in range(len(df_train)):
        optimizer.zero_grad()

        imgs, lbls = df_train[idx]
        
        out = model(imgs)
        lbls = lbls.squeeze_()
        lbls = lbls.to('cpu', dtype= torch.int64)
        #out = out.float()
        loss = criterion(out, lbls)
    
        epoch_loss.append(np.array(loss.item()))
        
        loss.backward()
        optimizer.step()
        
    epoch_loss = np.array(epoch_loss)
    mean_loss.append(np.mean(epoch_loss) / 100)
    print('Epoch:{0:3d}, Mean_Loss:{1:1.7f}'.format(e+1, mean_loss[-1]))

Epoch:  1, Mean_Loss:0.0135410
Epoch:  2, Mean_Loss:0.0134969
Epoch:  3, Mean_Loss:0.0131240
Epoch:  4, Mean_Loss:0.0123973
Epoch:  5, Mean_Loss:0.0113034
Epoch:  6, Mean_Loss:0.0092053
Epoch:  7, Mean_Loss:0.0136784
Epoch:  8, Mean_Loss:0.0135437
Epoch:  9, Mean_Loss:0.0104620
Epoch: 10, Mean_Loss:0.0078525
Epoch: 11, Mean_Loss:0.0073333
Epoch: 12, Mean_Loss:0.0068140
Epoch: 13, Mean_Loss:0.0069270
Epoch: 14, Mean_Loss:0.0065432
Epoch: 15, Mean_Loss:0.0061519
Epoch: 16, Mean_Loss:0.0061387


KeyboardInterrupt: 

In [15]:
def accuracy(df):
    wrong = 0
    for idx in range(len(df)):
        img, lbl = df[idx]
        if(model(img).item() != lbl.item()):
            wrong += 1

    return round(wrong / len(df), 3)

In [16]:
print(accuracy(df_test))

ValueError: only one element tensors can be converted to Python scalars